In [ ]:
import numpy as np

In [ ]:
import tensorflow as tf

In [ ]:
!nvidia-smi  #configuration of GPU

Mon Apr  7 14:27:29 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   56C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

! to run shell commands
%% for cell magics (affect the whole cell)

In [ ]:
%%writefile hello.c
#include <stdio.h>
#include <omp.h>

int main() {
    #pragma omp parallel
    {
        printf("Hello from thread %d\n", omp_get_thread_num());


    }
    return 0;
}

Writing hello.c


In [ ]:
!gcc -fopenmp hello.c -o hello

In [ ]:
!./hello

/bin/bash: line 1: ./hello: No such file or directory


In [ ]:
%%writefile hello.c
#include <stdio.h>
#include <omp.h>

int main() {
    omp_set_num_threads(4);  // Set number of threads to 4

    #pragma omp parallel
    {
        printf("Hello from thread %d\n", omp_get_thread_num());
    }

    return 0;
}


Writing hello.c


In [ ]:
!gcc -fopenmp hello.c -o hello #compile

In [ ]:
!./hello #run the code

Hello from thread 1
Hello from thread 3
Hello from thread 2
Hello from thread 0


In [ ]:
!lscpu | grep '^CPU(s):' #to get number of cores


CPU(s):                               2


In [ ]:
import os  #alternate method to know numbe of cores
os.cpu_count()


2

In [ ]:
%%writefile hello_openmp.c
#include <stdio.h>
#include <omp.h>

int main()
{

    #pragma omp parallel
    {
                int ID = omp_get_thread_num(); //Assigned once per thread
        printf("hello(%d)", ID); // Uses the thread's ID
        printf("world(%d) \n", ID); //Uses the same ID
    }
    return 0;

}


Overwriting hello_openmp.c


In [ ]:
!gcc -fopenmp hello_openmp.c -o hello_openmp

In [ ]:
!./hello_openmp

hello(1)world(1) 
hello(0)world(0) 


In [ ]:
%%writefile shared_var.c
#include <stdio.h>
#include <omp.h>

int main() {
    int sharedVar = 0;

    #pragma omp parallel shared(sharedVar)
    {
        #pragma omp critical
        sharedVar += 1;
    }

    printf("Final value of sharedVar: %d\n", sharedVar);
    return 0;
}

Writing shared_var.c


In [ ]:
%%writefile shared_var.c
#include <stdio.h>
#include <omp.h>

int main() {
    omp_set_num_threads(100);
    int sharedVar = 0;

    #pragma omp parallel shared(sharedVar)
    {
         #pragma omp critical
        sharedVar += 1;
    }

    printf("Final value of sharedVar: %d\n", sharedVar);
    return 0;
}

Overwriting shared_var.c


In [ ]:
!gcc -fopenmp shared_var.c -o shared_var

In [ ]:
!./shared_var

Final value of sharedVar: 100


In [ ]:
%%writefile matrix_addition.c
#include <stdio.h>
#include <omp.h>

#define N 3

int main() {
    int A[N][N] = {{1, 2, 3}, {4, 5, 6}, {7, 8, 9}};
    int B[N][N] = {{9, 8, 7}, {6, 5, 4}, {3, 2, 1}};
    int C[N][N];

    #pragma omp parallel for collapse(2)
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            C[i][j] = A[i][j] + B[i][j];
        }
    }

    printf("Resultant Matrix:\n");
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            printf("%d ", C[i][j]);
        }
        printf("\n");
    }

    return 0;
}


Writing matrix_addition.c


In [ ]:
!gcc -fopenmp matrix_addition.c -o matrix_addition


In [ ]:
!./matrix_addition


Resultant Matrix:
10 10 10 
10 10 10 
10 10 10 


In [ ]:
%%writefile matrix_add_timed.c
#include <stdio.h>
#include <omp.h>

#define N 3

int main() {
    int A[N][N] = {{1, 2, 3}, {4, 5, 6}, {7, 8, 9}};
    int B[N][N] = {{9, 8, 7}, {6, 5, 4}, {3, 2, 1}};
    int C[N][N];

    double start_time = omp_get_wtime();  // Start timer-wall clocktime

    // Matrix addition (currently serial)
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            C[i][j] = A[i][j] + B[i][j];
        }
    }

    printf("Resultant Matrix:\n");
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            printf("%d ", C[i][j]);
        }
        printf("\n");
    }

    double end_time = omp_get_wtime();  // End timer
    printf("Execution Time = %f seconds\n", end_time - start_time);

    return 0;
}


Writing matrix_add_timed.c


In [ ]:
!gcc -fopenmp matrix_add_timed.c -o matrix_add_timed


In [ ]:
!./matrix_add_timed


Resultant Matrix:
10 10 10 
10 10 10 
10 10 10 
Execution Time = 0.000046 seconds


In [ ]:
%%writefile parallel_sum.c
#include <stdio.h>
#include <omp.h>
#define SIZE 1000000

int main() {
    int i;
    double sum = 0.0;
    double array[SIZE];

    // Initialize array with values
    for (i = 0; i < SIZE; i++) {
        array[i] = 1.0; // Simple case where sum = SIZE
    }

    double start_time = omp_get_wtime();  // Start timer

    #pragma omp parallel for reduction(+:sum)
    for (i = 0; i < SIZE; i++) {
        sum += array[i];
    }

    double end_time = omp_get_wtime();    // End timer

    printf("Sum = %.2f\n", sum);
    printf("Execution Time = %f seconds\n", end_time - start_time);

    return 0;
}


Writing parallel_sum.c


In [ ]:
!gcc -fopenmp parallel_sum.c -o parallel_sum


In [ ]:
!./parallel_sum


Sum = 1000000.00
Execution Time = 0.001926 seconds


In [ ]:
%%writefile shared_var_critical.c
#include <stdio.h>
#include <omp.h>

int main() {
    int sharedVar = 0;

    // Optional: Set number of threads
    omp_set_num_threads(8);  // You can change this value

    #pragma omp parallel shared(sharedVar)
    {
        #pragma omp critical
        sharedVar += 1;
    }

    printf("Final value of sharedVar: %d\n", sharedVar);
    return 0;
}



Overwriting shared_var_critical.c


In [ ]:
!gcc -fopenmp shared_var_critical.c -o shared_var_critical




In [ ]:
!./shared_var_critical


Final value of sharedVar: 8


In [ ]:
%%writefile omp_single_example.c
#include <stdio.h>
#include <omp.h>

void add(int a, int b, int *result) {
    #pragma omp parallel
    {
        #pragma omp single
        {
        *result = a + b;
        }
}
}

int main() {
    int x = 5, y = 10, sum = 0;
    add(x, y, &sum);
    printf("Sum: %d\n", sum);
    return 0;
}


Overwriting omp_single_example.c


In [ ]:
!gcc -fopenmp omp_single_example.c -o omp_single_example


In [ ]:
!./omp_single_example


Sum: 15


In [ ]:
%%writefile modified.c
#include <stdio.h>
#include <omp.h>

void modify(int a) {
    #pragma omp parallel
    {
        // Each thread will print its own copy of 'a'
        printf("Thread %d: Original a = %d\n", omp_get_thread_num(), a);
        // Modify 'a' inside the thread
        a += 10;
        // Show modified value (thread-local)
        printf("Thread %d: Modified a = %d\n", omp_get_thread_num(), a);
    }
}

int main() {
    int x = 5;
    modify(x);
    printf("Back in main: x = %d\n", x); // Still 5
    return 0;
}


Writing modified.c


In [ ]:
!gcc -fopenmp modified.c -o output

In [ ]:
!./output

Thread 0: Original a = 5
Thread 0: Modified a = 15
Thread 1: Original a = 5
Thread 1: Modified a = 25
Back in main: x = 5


In [ ]:
%%writefile newfile.c
#include <stdio.h>
#include <omp.h>

void add(int a, int b, int *result) {
    #pragma omp parallel
    {
        printf("Thread %d: Inside parallel region\n", omp_get_thread_num());

        // All threads execute this
        *result = a + b;
        a=a+10;
        b=b+10;
        printf("Thread %d: Calculated result = %d\n", omp_get_thread_num(), *result);
    }
}

int main() {
    int x = 5, y = 10, sum = 0;
    add(x, y, &sum);
    printf("Back in main: sum = %d\n", sum);
    return 0;
}


Overwriting newfile.c


In [ ]:
!gcc -fopenmp newfile.c -o output1

In [ ]:
!./output1

Thread 0: Inside parallel region
Thread 0: Calculated result = 15
Thread 1: Inside parallel region
Thread 1: Calculated result = 35
Back in main: sum = 35


In [ ]:
%%writefile example.c
#include <stdio.h>
#include <omp.h>

int main() {
    #pragma omp parallel
    {
        #pragma omp critical
        {
            // Every thread prints (one at a time)
            printf("In critical by thread %d\n", omp_get_thread_num());
        }

        #pragma omp single
        {
            // Only one thread prints this
            printf("In single by thread %d\n", omp_get_thread_num());
        }
    }
    return 0;
}


Overwriting example.c


In [ ]:
!gcc -fopenmp example.c -o output2

In [ ]:
!./output2

In critical by thread 0
In critical by thread 1
In single by thread 0
